In [2]:
class StockData():
    
    def __init__(self, closing_price, date_ddmmyyyy):
        '''
        Initialize the instance with an iterable of closing_price and date_ddmmyyyy. Returns None
        
        '''
        self.closing_price= closing_price
        self.date_ddmmyyyy = date_ddmmyyyy
        return None
        
        
    def YearlyChange(self):
        '''
        This function takes closing price and date and then returns an iterable of % yaerly changes
        
        '''
        assert len(self.closing_price) == len(self.date_ddmmyyyy), 'The length of date and closing_price are not equal'
        yearly_percent_change = []
        i, j = 0, 0
        while( i < len(self.closing_price)):
            #print(i)
            #j = i
            final_closing_price = self.closing_price[i]
            while(j+1<len(self.closing_price)):
                    #print(i)
                if(self.date_ddmmyyyy[j][7:] == self.date_ddmmyyyy[j+1][7:]):
                    if(j == len(self.closing_price)):
                        pass
                    else:
                        j = j+1
                else:
                    #print(j)
                    break
                #i = j
            if (j<len(self.closing_price)-1):
                initial_closing_price = self.closing_price[j+1]
            else:
                initial_closing_price = self.closing_price[j]
            yearly_percent_change.append((final_closing_price - initial_closing_price)*100/initial_closing_price)                
            i = j+1
            j = j+1
            if(j == len(self.closing_price)-1):
                break
                  
        return yearly_percent_change
    
    
    def MonthlyChange(self):
        assert len(self.closing_price) == len(self.date_ddmmyyyy), 'The length of date and closing_price are not equal'
        monthly_percent_change = []
        i, j = 0, 0
        while( i < len(self.closing_price)):
            #print(i)
            #j = i
            final_closing_price = self.closing_price[i]
            while(j+1<len(self.closing_price)):
                    #print(i)
                if(self.date_ddmmyyyy[j][3:5] == self.date_ddmmyyyy[j+1][3:5]):
                    if(j == len(self.closing_price)):
                        pass
                    else:
                        j = j+1
                else:
                    #print(j)
                    break
                #i = j
            if (j<len(self.closing_price)-1):
                initial_closing_price = self.closing_price[j+1]
            else:
                initial_closing_price = self.closing_price[j]
            monthly_percent_change.append((final_closing_price - initial_closing_price)*100/initial_closing_price)                
            i = j+1
            j = j+1
            if(j == len(self.closing_price)-1):
                break
        return monthly_percent_change

In [3]:
import pandas as pd
import numpy as np

In [5]:
df_nifty_safe_haven = pd.read_csv("C://Users/Prithvi Krishna/Documents/Safe Haven/Nifty Data for Safe Haven.csv")

In [22]:
nifty_50 = StockData(df_nifty_safe_haven['Close'], df_nifty_safe_haven['Date'])

In [228]:
investment = 1
import numpy as np
import random
ch_array = np.array(nifty_50.YearlyChange())

In [229]:
for i in range(100):
    investment = investment * (1 + 0.01 * random.choice(ch_array))
investment

7507.68797975853

In [10]:
len(nifty_50.MonthlyChange())

232

In [30]:
class IndexRandomWalk(StockData):
    
    def __init__(self, investment_horizon, num_of_iterations, year =1, *args):
        #assert type(self.change_data) is list, "Change Data provided is not in a list" 
        #self.change_data = change_data
        self.year = year
        self.investment_horizon = investment_horizon
        self.num_of_iterations = num_of_iterations
        super().__init__(*args)
        
    def GeometricWalkIndex(self):
        import random
        import math
        import statistics
        import numpy as np
        #final_investment = 1
        collection_iter = []
        if(self.year == 1):
            change_array = np.array(self.YearlyChange())
            for i in range(self.num_of_iterations):
                final_investment = 1
                for i in range(self.investment_horizon):
                    final_investment = final_investment*(1+ 0.01*random.choice(change_array))
            
                collection_iter.append(final_investment)
        else:
            change_array = np.array(self.MonthlyChange())
            for i in range(self.num_of_iterations):
                final_investment = 1
                for i in range(self.investment_horizon*12):
                    final_investment = final_investment*(1+ 0.01*random.choice(change_array))

                collection_iter.append(final_investment)
                
        median_final_investment = statistics.median(collection_iter)
        fifth_percentile_investment = np.percentile(np.array(collection_iter), 5)
        return collection_iter
        #return median_final_investment, fifth_percentile_investment
    
    
    def GeometricWalkIndexFixed(self, fixed_rate):
        self.fixed_rate = fixed_rate
        import numpy as np
        import statistics
        #dict_median_fifth = dict()
        median_list = []
        fifth_perc_list = []
        if(self.year == 1):
            change_array = np.array(self.YearlyChange())
            for j in range(1,101):
                collection_iter = []
                fraction = j/100
                for i in range(self.num_of_iterations):
                    final_investment = 1
                    for i in range(self.investment_horizon):
                        final_investment = fraction * final_investment*(1+ 0.01*random.choice(change_array)) + (1-fraction)*(1+self.fixed_rate)

                    collection_iter.append(final_investment)
                median_final_investment = statistics.median(collection_iter)
                median_list.append(median_final_investment)
                fifth_percentile_investment = np.percentile(np.array(collection_iter), 5)
                fifth_perc_list.append(fifth_percentile_investment)
        else:
            change_array = np.array(self.MonthlyChange())
            for j in range(1,101):
                collection_iter = []
                fraction = j/100
                for i in range(self.num_of_iterations):
                    final_investment = 1
                    for i in range(self.investment_horizon*12):
                        final_investment = fraction * final_investment*(1+ 0.01*random.choice(change_array)) + (1-fraction)*(1+self.fixed_rate)

                    collection_iter.append(final_investment)
                median_final_investment = statistics.median(collection_iter)
                median_list.append(median_final_investment)
                fifth_percentile_investment = np.percentile(np.array(collection_iter), 5)
                fifth_perc_list.append(fifth_percentile_investment)
        return median_list, fifth_perc_list
    

    
    def GeometricWalkIndexInsurance(self, payoff, index_returns_bucket):
        self.payoff = payoff
        self.index_returns_bucket = index_returns_bucket
        #GeometricWalkIndex(self):
        import random
        import math
        import statistics
        import numpy as np
        #final_investment = 1
        median_list = []
        fifth_perc_list = []
        median_returns_list = []
        fifth_perc_returns_list = []
        if(self.year == 1):
            change_array = np.array(self.YearlyChange())
            for j in range(1,101):
                collection_iter = []
                fraction = j/100
                for i in range(self.num_of_iterations):
                    final_investment = 1
                    for i in range(self.investment_horizon):
                        index_returns = random.choice(change_array)
                        if (index_returns <= index_returns_bucket):
                            final_investment = fraction * final_investment*(1+ 0.01*index_returns) + (1-fraction)*(1+self.payoff* fraction)
                        else:
                            final_investment = fraction * final_investment*(1+ 0.01*index_returns)

                    collection_iter.append(final_investment)
                median_final_investment = statistics.median(collection_iter)
                median_list.append(median_final_investment)
                median_returns_list.append((median_final_investment)**(1/self.investment_horizon)-1)
                
                fifth_percentile_investment = np.percentile(np.array(collection_iter), 5)
                fifth_perc_list.append(fifth_percentile_investment)
                fifth_perc_returns_list.append((fifth_percentile_investment)**(1/self.investment_horizon)-1)
        else:
            change_array = np.array(self.MonthlyChange())
            for j in range(1,101):
                collection_iter = []
                fraction = j/100
                for i in range(self.num_of_iterations):
                    final_investment = 1
                    for i in range(self.investment_horizon*12):
                        index_returns = random.choice(change_array)
                        if (index_returns <= index_returns_bucket):
                            final_investment = fraction * final_investment*(1+ 0.01*index_returns) + (1-fraction)*(1+self.payoff* fraction)
                        else:
                            final_investment = fraction * final_investment*(1+ 0.01*index_returns)

                    collection_iter.append(final_investment)
                median_final_investment = statistics.median(collection_iter)
                median_list.append(median_final_investment)
                median_returns_list.append((median_final_investment)**(1/self.investment_horizon*12)-1)
                
                fifth_percentile_investment = np.percentile(np.array(collection_iter), 5)
                fifth_perc_list.append(fifth_percentile_investment)
                fifth_perc_returns_list.append((fifth_percentile_investment)**(1/self.investment_horizon*12)-1)
        return median_list, fifth_perc_list, median_returns_list, fifth_perc_returns_list
    
    
    #
    #def GeometricWalkIndexInsurance
    

In [189]:
type(nifty_50.MonthlyChange()) is list

True

In [31]:
nifty_random_walk = IndexRandomWalk(25, 20000, 0, df_nifty_safe_haven['Close'], df_nifty_safe_haven['Date'])

In [32]:
median_andfith_perc_of_walk_monthly = nifty_random_walk.GeometricWalkIndex()

In [38]:
import statistics
(np.percentile(np.array(median_andfith_perc_of_walk_monthly),5)**(1/300))**12

1.0526635740663393

In [29]:
nifty_random_walk.MonthlyChange()

[3.814934292538566,
 1.1567240803963346,
 6.504637382015015,
 -0.4056988434860174,
 1.1119026233468654,
 6.560881874055705,
 -2.482879467877767,
 7.809421734219033,
 11.394128358414083,
 3.5105422958777726,
 -1.2289791437980306,
 2.8360628349791552,
 15.585628842520606,
 -2.8357285570847615,
 14.680003489284983,
 -23.246367755038275,
 -6.356325394370556,
 -1.695778837896366,
 0.9323119927339507,
 1.503689765058986,
 3.5121509091939047,
 4.0931667158052365,
 -0.8522216225939917,
 -6.750092260207328,
 1.4866170418321152,
 1.0689183492631562,
 7.703497799397727,
 -0.35500117718206364,
 -0.290907751863039,
 -0.1305537040016616,
 4.719061097953128,
 -4.975549954484951,
 -6.421386070801757,
 2.8529916787742704,
 5.77814207141294,
 -0.029796961641074437,
 6.186163323017289,
 -3.613412943099345,
 -4.850059395884911,
 4.719534313958236,
 2.9741212823484116,
 -1.0522190937853764,
 5.585068344809256,
 -1.3037034049546705,
 -1.5798195909537538,
 4.737949850591143,
 3.409268006943221,
 1.4203569968

In [18]:
median_andfith_perc_of_walk_monthly[0]**(1/(nifty_random_walk.investment_horizon * 12))

1.0108487360870686

In [283]:
median_andfith_perc_of_walk_monthly

(18.640550511608655, 1.5909573028773312)

In [272]:
median_returns = 100*(median_andfith_perc_of_walk[0]**(.04)-1)
median_returns

12.44134141015596

In [271]:
fifth_perc_returns = 100*(median_andfith_perc_of_walk[1]**(0.04)-1)
fifth_perc_returns

2.0987625077099725

In [311]:
nifty_random_walk.GeometricWalkIndexFixed(0.16)

([1.16,
  1.1616597870907062,
  1.1629995030988347,
  1.1648945275382587,
  1.1665797696225528,
  1.16871392956108,
  1.170136274301758,
  1.1722706116546338,
  1.173805879698366,
  1.1759345683501574,
  1.1776437957353836,
  1.1801764480657544,
  1.1821289610157169,
  1.1852458171891378,
  1.1872714607480832,
  1.1897375890623334,
  1.1915650204537758,
  1.1941537144326693,
  1.197051565962651,
  1.1985654464431694,
  1.202581624832278,
  1.2051705248292233,
  1.207792847788307,
  1.2109956889296378,
  1.2147961209825602,
  1.2178514330958898,
  1.2205897839171238,
  1.224588546265033,
  1.2266575020582553,
  1.2329462604015102,
  1.233764974068,
  1.2408355462396883,
  1.2442588118556002,
  1.247310659571996,
  1.2525266064686036,
  1.25762926956995,
  1.2629129129966765,
  1.2667000482280404,
  1.2728257795144788,
  1.2771950002409982,
  1.2820044778992863,
  1.2902008235284175,
  1.2948270543788527,
  1.301187012795555,
  1.312273165151792,
  1.3169815658361264,
  1.320457253237116

In [39]:
z = nifty_random_walk.GeometricWalkIndexInsurance(10, -10)

In [332]:
z[2:]

([-0.5134259072574362,
  -0.4484657050679822,
  -0.4120194328909358,
  -0.3848163110182743,
  -0.3597511504640858,
  -0.337603803331207,
  -0.32336638333980505,
  -0.30749610561061236,
  -0.2860665077600225,
  -0.2757905926426235,
  -0.26314832514444453,
  -0.2539982718167443,
  -0.24012278297826772,
  -0.23137268948726175,
  -0.21846445333078046,
  -0.2151658037684928,
  -0.2075910898462746,
  -0.19497403971997473,
  -0.19228962920817716,
  -0.18037756795069793,
  -0.17581262736764636,
  -0.16915642825513433,
  -0.16218222134880633,
  -0.15533443363009125,
  -0.14787939611235357,
  -0.14637643479713447,
  -0.13501174934222004,
  -0.13139407448664697,
  -0.12598595031429582,
  -0.11975161919024735,
  -0.11197977041440232,
  -0.10914284180108147,
  -0.10028521883854025,
  -0.10165339284101904,
  -0.08921607190531222,
  -0.0886339997044635,
  -0.08681034172281155,
  -0.08093189028443326,
  -0.08012991162847416,
  -0.07676154992908069,
  -0.06932752612473181,
  -0.06352222811583152,
  -0.

In [ ]:
from matplotlib import pyplot